In [ ]:
# import tensorflow as tf
# import tensorflow_recommenders as tfrs
# import pandas as pd

# # Crear un DataFrame con las características de los ítems
# items_df = pd.DataFrame({
#     'item_id': [1, 2, 3, 4],
#     'genre': ['Action', 'Comedy', 'Drama', 'Adventure'],
#     'price': [10.99, 7.99, 12.49, 9.99]
# })

# # Convertir características categóricas en variables numéricas
# items_df = pd.get_dummies(items_df, columns=['genre'], drop_first=True)

# # Convertir DataFrame en TensorFlow Dataset
# items_ds = tf.data.Dataset.from_tensor_slices(dict(items_df))

# # Mostrar una muestra del dataset
# for x in items_ds.take(1):
#     print(x)

# #################################################
# class ItemModel(tf.keras.Model):

#     def __init__(self):
#         super().__init__()
#         # Embedding layers for item features
#         self.genre_embedding = tf.keras.Sequential([
#             tf.keras.layers.Dense(32, activation='relu'),
#             tf.keras.layers.Dense(16)
#         ])
#         self.price_embedding = tf.keras.Sequential([
#             tf.keras.layers.Dense(32, activation='relu'),
#             tf.keras.layers.Dense(16)
#         ])

#     def call(self, inputs):
#         # Concatenate the item features
#         return tf.concat([
#             self.genre_embedding(inputs['genre_Action']),
#             self.genre_embedding(inputs['genre_Comedy']),
#             self.genre_embedding(inputs['genre_Drama']),
#             self.genre_embedding(inputs['genre_Adventure']),
#             self.price_embedding(inputs['price'])
#         ], axis=1)


# class UserModel(tf.keras.Model):
#     def __init__(self):
#         super().__init__()
#         self.user_embedding = tf.keras.Sequential([
#             tf.keras.layers.Dense(32, activation='relu'),
#             tf.keras.layers.Dense(16)
#         ])

#     def call(self, inputs):
#         return self.user_embedding(inputs['user_id'])


# class RecommenderModel(tfrs.models.Model):
#     def __init__(self):
#         super().__init__()
#         self.query_model = UserModel()
#         self.candidate_model = ItemModel()
#         self.task = tfrs.tasks.Retrieval(
#             metrics=tfrs.metrics.FactorizedTopK(
#                 candidates=items_ds.batch(128).map(self.candidate_model)
#             )
#         )

#     def compute_loss(self, features, training=False):
#         query_embeddings = self.query_model({'user_id': features['user_id']})
#         candidate_embeddings = self.candidate_model({
#             'genre_Action': features['genre_Action'],
#             'genre_Comedy': features['genre_Comedy'],
#             'genre_Drama': features['genre_Drama'],
#             'genre_Adventure': features['genre_Adventure'],
#             'price': features['price']
#         })

#         return self.task(query_embeddings, candidate_embeddings)

# # Crear el dataset de entrenamiento
# user_interactions = pd.DataFrame({
#     'user_id': [1, 1, 2, 2, 3],
#     'item_id': [1, 2, 1, 3, 4]
# })

# interactions_ds = tf.data.Dataset.from_tensor_slices(dict(user_interactions))

# # Fusionar datos de interacción con características de ítems
# interactions_ds = interactions_ds.map(lambda x: {**x, **items_df.loc[items_df['item_id'] == x['item_id'].numpy()].to_dict('records')[0]})

# # Dividir los datos en entrenamiento y prueba
# tf.random.set_seed(42)
# shuffled = interactions_ds.shuffle(len(interactions_ds), seed=42, reshuffle_each_iteration=False)
# train_size = int(0.8 * len(shuffled))
# train = shuffled.take(train_size)
# test = shuffled.skip(train_size)

# # Crear y compilar el modelo
# model = RecommenderModel()
# model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# # Entrenar el modelo
# model.fit(train.batch(4), epochs=10)

# # Evaluar el modelo
# model.evaluate(test.batch(4), return_dict=True)

# #################################################

# # Crear un dataset con los usuarios para los que queremos hacer recomendaciones
# user_ids = tf.data.Dataset.from_tensor_slices([1])

# # Hacer recomendaciones
# scann_index = tfrs.layers.factorized_top_k.ScaNN(model.query_model)
# scann_index.index_from_dataset(items_ds.batch(128).map(model.candidate_model))

# for user_id in user_ids:
#     scores, ids = scann_index({
#         'user_id': tf.constant([user_id])
#     })
#     print(f"Recomendaciones para el usuario {user_id}: {ids.numpy()}")
